<a href="https://colab.research.google.com/github/Knilik97/neurocommunication_prep/blob/main/PetProject_Neyro_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Надо продумать простую базу знаний например парсер в файл txt тема какая будет чата бота ?
Может сосредоточиться на более простых вещах ? ну типа что нужно работодателю!

И тут я придумал, обучения такого класса рабочих, что бы человек знал свои права и обязанности,
думаю, что будет полезно студентам и людям, которые хотят зарабатывать на этих должностях, смогли кратко и точно узнать, свои обязанности,
дабы ничего лишнего не стали требовать. Гражданин РФ должен уважать свои и чужие права человека.

Может нейро - преподаватель ? тогда что именно он будет консультировать ?
Чему именно он будет обучать ? повышение знаний класса рабочих, например (разнорабочий, грузчик, кладовщик, заведующий складом)

	Структура файла:

* Должность название
* кратко обязанности
* что он должен делать
* чего он не должен делать
* ссылка на ТК РФ

Ну для начала думаю самый шик, осталось определиться с загрузкой контурной модели.

Представим такую ситуацию:

Я работодатель, нанимаю сотрудников, имеется большой объём текста с ознакомлением своих обязанностей и прочего, как правило мало кто читает, особенно студенты,
поэтому, можно предложить им ознакомиться со своими обязанностями у нейро-сотрудника, сухо, без воды и лишней прочей информации. Ну так сыровато, но в целом я считаю для практики это хорошим началом идеи реализации.


# LlamaIndex, как высказалась мой куратор, этот фреймворк как вьючная змея, и ей нужны правильные зависимости, установлю через файл requirements.txt

In [ ]:
%%writefile requirements.txt
transformers>=4.42.0
llama_index==0.10.46
pyvis==0.3.2
Ipython==7.34.0
langchain==0.2.5
pypdf==4.2.0
langchain_community==0.2.5
llama-index-llms-huggingface==0.2.3
llama-index-embeddings-huggingface==0.2.2
llama-index-embeddings-langchain==0.1.2
langchain-huggingface==0.0.3
sentencepiece==0.1.99
accelerate==0.31.0
bitsandbytes==0.43.1
peft==0.11.1
llama-index-readers-wikipedia==0.1.4
wikipedia==1.4.0
triton<=3.0 # Ещё необходимый модуль для загрузки модели

huggingface-hub==0.23.3
torch==2.3.1
packaging==24.1
pyyaml==6.0.1
requests==2.31.0
tqdm==4.66.4
filelock==3.14.0
regex==2024.5.15
typing-extensions==4.12.2
safetensors==0.4.3
tokenizers==0.19.1

Overwriting requirements.txt


In [ ]:
!pip install -r requirements.txt

In [ ]:
# Импортируем классы и функции из модуля llama_index.core
from llama_index.core import SimpleDirectoryReader
from llama_index.core import Settings
from llama_index.core import StorageContext

# Импортируем классы для работы с эмбеддингами и языковыми моделями из HuggingFace
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM

# Импортируем классы и функции из библиотеки PEFT для адаптации моделей
from peft import PeftModel, PeftConfig

# Импортируем классы и функции из библиотеки Transformers для работы с языковыми моделями
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, BitsAndBytesConfig

# Импортируем библиотеку PyTorch для работы с тензорами и моделями
import torch

# Импортируем класс для работы с эмбеддингами из Langchain
from llama_index.embeddings.langchain import LangchainEmbedding

# Импортируем класс для работы с эмбеддингами из Langchain и Hugging Face
from langchain_huggingface import HuggingFaceEmbeddings

Конектимся к платформе Hugging_Face

In [ ]:
from google.colab import userdata
userdata.get('huggingface')

from huggingface_hub import login
import os

# Получение токена из секретов Colab
hf_token = os.getenv("huggingface")

# Аутентификация
login(token=hf_token)

In [ ]:
# Вспомогательная ф-ция для модели
def messages_to_prompt(messages):
    # Инициализируем пустую строку для хранения итогового промпта
    prompt = ""

    # Проходим по каждому сообщению в списке сообщений
    for message in messages:
        # Если роль сообщения 'system', добавляем его в промпт с соответствующими тегами
        if message.role == 'system':
            prompt += f"<s>{message.role}\n{message.content}</s>\n"
        # Если роль сообщения 'user', добавляем его в промпт с соответствующими тегами
        elif message.role == 'user':
            prompt += f"<s>{message.role}\n{message.content}</s>\n"
        # Если роль сообщения 'bot', добавляем только начальный тег для бота
        elif message.role == 'bot':
            prompt += f"<s>bot\n"

    # Проверяем, начинается ли промпт с системного сообщения, если нет, добавляем пустое системное сообщение
    if not prompt.startswith("<s>system\n"):
        prompt = "<s>system\n</s>\n" + prompt

    # Добавляем финальный начальный тег для бота в конец промпта
    prompt = prompt + "<s>bot\n"

    # Возвращаем итоговый промпт
    return prompt

def completion_to_prompt(completion):
    # Формируем промпт из завершения, добавляя теги для системного и пользовательского сообщений
    return f"<s>system\n</s>\n<s>user\n{completion}</s>\n<s>bot\n"


# Загрузка модели

Буду использовать русскоязычную версию saiaga_7b, так же буду квантовать, что бы хватило вычеслительных ресурсов

In [ ]:
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer
#для настройки и управления параметрами обучения моделей машинного обучения с использованием методов эффективного обучения (Parameter-Efficient Fine-Tuning, PEFT).
from peft import PeftConfig, PeftModel

# Определяем параметры квантования, иначе модель не выполниться в колабе
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Задаем имя модели
MODEL_NAME = "IlyaGusev/saiga_mistral_7b"

# Создание конфига, соответствующего методу PEFT (в нашем случае LoRA)
config = PeftConfig.from_pretrained(MODEL_NAME)

# Загружаем базовую модель, ее имя берем из конфига для LoRA
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,          # идентификатор модели
    quantization_config=quantization_config, # параметры квантования
    torch_dtype=torch.float16,               # тип данных
    device_map="auto"                        # автоматический выбор типа устройства
)

# Загружаем LoRA модель
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)

# Переводим модель в режим инференса
# Можно не переводить, но явное всегда лучше неявного
model.eval()

# Загружаем токенизатор
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

...

adapter_model.safetensors:   0%|          | 0.00/54.6M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Автор позаботился о конфигурации модели, взгляну на неё

In [ ]:
# Загружаем конфигурацию генерации из предобученной модели.
# MODEL_NAME - это строка, содержащая имя модели, из которой мы хотим получить конфигурацию.
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

generation_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}



# Сохраняю квантованную весрию модели себе на диск, что бы не квантовать её каждый раз

In [ ]:
# Указываем путь для сохранения квантованной модели
SAVE_PATH = "quantized_model_saiga"

# Сохраняем модель с настройками квантования
model.save_pretrained(
    SAVE_PATH,
    safe_serialization=False  # Важно для сохранения квантованной версии
)

# Сохраняем токенизатор
tokenizer.save_pretrained(SAVE_PATH)


('quantized_model_saiga/tokenizer_config.json',
 'quantized_model_saiga/special_tokens_map.json',
 'quantized_model_saiga/tokenizer.model',
 'quantized_model_saiga/added_tokens.json')

# Загрузка квантованной версии модели

In [ ]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer

# Загружаем квантованную модель
model = AutoModelForCausalLM.from_pretrained(
    "quantized_model_saiga",  # путь к сохраненной модели
    load_in_4bit=True,  # указываем, что модель уже квантована
    torch_dtype=torch.float16,
    device_map="auto"
)

# Загружаем токенизатор
tokenizer = AutoTokenizer.from_pretrained("quantized_model", use_fast=False)

# Переводим модель в режим инференса
model.eval()

Передача в класс ранее объявленные вспомогательные функции.

In [ ]:
llm = HuggingFaceLLM(
    model=model,             # модель
    model_name=MODEL_NAME,   # идентификатор модели
    tokenizer=tokenizer,     # токенизатор
    max_new_tokens=generation_config.max_new_tokens, # параметр необходимо использовать здесь, и не использовать в generate_kwargs, иначе ошибка двойного использования
    model_kwargs={"quantization_config": quantization_config}, # параметры квантования
    generate_kwargs = {   # параметры для инференса
      "bos_token_id": generation_config.bos_token_id, # токен начала последовательности
      "eos_token_id": generation_config.eos_token_id, # токен окончания последовательности
      "pad_token_id": generation_config.pad_token_id, # токен пакетной обработки (указывает, что последовательность ещё не завершена)
      "no_repeat_ngram_size": generation_config.no_repeat_ngram_size,
      "repetition_penalty": generation_config.repetition_penalty,
      "temperature": generation_config.temperature,
      "do_sample": True,
      "top_k": 50,
      "top_p": 0.95
    },
    messages_to_prompt=messages_to_prompt,     # функция для преобразования сообщений к внутреннему формату
    completion_to_prompt=completion_to_prompt, # функции для генерации текста
    device_map="auto",                         # автоматически определять устройство
)

# Загрузка и обработка данных для модели

In [ ]:
# Загрузка текстового файла (.txt)

# Чтение содержимого текстового файла
file_path = '/content/NeyroWork.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read(500)  # Прочитает первые 100 символов
print(text)

Гру́зчик — это профессия, в которой используется преимущественно только мускульная физическая сила человека для выполнения разных рабочих операций по перемещению грузов: погрузки, выгрузки, кантования, перекатывания и подъёма.

В ряде случаев также используются приспособления для облегчения тяжёлого физического труда. Название профессии происходит от слова груз (тяжесть). Профессия грузчика чрезвычайно широко распространена по всем странам и континентам, и в мире трудятся десятки миллионов грузч


**Выше я специально вывел первые 500 символов, посмотреть что же в файле, т.к. буду применять запросы к модели, кто такой грузчик**

In [ ]:
from llama_index.core import SimpleDirectoryReader
import os

file_path = '/content/reader/NeyroWork.txt'

# Проверяем существование файла
if os.path.exists(file_path):
    print("Файл найден")
    # Правильно загружаем документ через ридер
    documents = SimpleDirectoryReader(
        input_files=[file_path]  # указываем список файлов
    ).load_data()
else:
    print(f"Файл не найден по пути: {file_path}")

Файл найден


In [ ]:
# Импортируем класс HuggingFaceEmbeddings из модуля langchain_huggingface
from langchain_huggingface import HuggingFaceEmbeddings

# Создаем объект embed_model, используя класс LangchainEmbedding
# В качестве параметра передаем объект HuggingFaceEmbeddings, который инициализируется с моделью
# "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Установка модели языковой модели (LLM)
Settings.llm = llm

# Установка модели для встраивания (embedding model)
Settings.embed_model = embed_model

# Установка размера фрагмента (chunk size) для обработки данных
Settings.chunk_size = 512

In [ ]:
# Импортируем необходимые классы из библиотеки llama_index
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

# Создаем индекс из документов
# Предполагается, что 'documents' - это переменная, содержащая список документов для индексации
index = VectorStoreIndex.from_documents(documents)

# Подготавливаем движок для выполнения запросов к индексу
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("Кто такой грузчик ? .")
print(response)

Грузчик - это работник, который занимается погрузкой и выгрузкой грузов. Он обычно работает на складах, транспортных компаниях, портах и других локациях, где требуется перемещение товаров и материалов. Грузчики могут использовать различные виды техники, включая краны, лебёдки, тележки и другие погрузочно-разгрузочные приспособления, чтобы выполнять свои задачи.


In [ ]:
response = query_engine.query("Чем отличаются обящаности кладовщика и заведущего склада ? .")
print(response)

Отличия между обязанностями кладовщика и заведующего складом заключаются в том, что заведующий складом имеет более широкий круг обязанностей, включая не только хранение и обработку товаров, но и организацию погрузочно-разгрузочных процессов, контроль за сохранностью и качеством товаров, поддержание информационного обеспечения склада, а также координацию действий других сотрудников склада. Кладовщик, в свою очередь, занимается только хранением и обработкой товаров, без участия в организации погрузочно-разгрузочных процеслов.


In [ ]:
response = query_engine.query("Кто такой грузчик ? Расскажи операясь на файл.")
print(response)

Грузчик - это человек, который занимается перемещением грузов. Он может работать на различных видах транспорта, в том числе на железнодорожном, морском, воздушном, автомобильном, а также на складах и в других подобных организациях. Грузчики могут быть как специалисты с большим опытом работы, так и люди, которые только начали свою трудовую деятельность. Они обычно работают в командах, где каждый член отвечает за свою часть работы. Грузчики часто используют различные инструменты и оборудование для перемещения грузов, включая краны, тележки, подъемники и т. д.


In [ ]:
response = query_engine.query("Расскажи про должностные обзяности грузчика ?.")
#print(response)

Ответственность грузчика:

* Невыполнение и/или несвоевременное, нехалатное выполнение своих должностных обязанности;
* Нарушение действующих инструкций, приказов и распоряджений по сохранению коммерческой тайны и секретной информации;
* Нарушение правил внутреннего трудового распорядка и трудовой дисциплины;
* Нарушение правил техники безопасности и противопожерной безопасности.


# Вывод про первую настройку модели

И так, вижу модель галюцианирует, вопросы про грузчика в целом хорошие, а вот про должностные обязаности, модель начала сразу с Ответсвенности грузичка, хотя в файлы можно наблюдать следующее:


Должностные обязанности грузчика
Грузчик выполняет следующие должностные обязанности:

2.1. Осуществляет погрузку и выгрузку.

2.2. Выполняет внутрискладскую переработку грузов - сортировку, укладку, переноску, перевеску, фасовку вручную или с применением простейших погрузочно-разгрузочных приспособлений и средств транспортировки (тачек, тележек, транспортеров и других подъемно-транспортных механизмов).

2.3. Устанавливает лебедки, подъемные блоки, устройство временных скатов и
и т.д



# Улучшение RAG

In [ ]:
# Начну с улучшенного промта

response = query_engine.query("Расскажи про должностные обзяности грузчика ? Опираясь на контекст документа, если сомвневаешься в ответе, то ничего не придумывай.")
print(response)

Для выполнения своих обязанностей грузчик должен:

1. Вместе с другими сотрудниками выполнять задачи по приему, хранению и отгрузке товаров, материалов, оборудования и другой товарно-материальной ценности.
2. Использовать технические средства и механизмы для перемещения грузов, согласно требованиям технических установок и инструкций по эксплуатации.
3. Соблюдать правила техники безопасности и противопожарной борьбы, а также соответствовать требованиям законодательства в области охраны труда.
4. Учитывать индивидуальные особенности груза (размер, вес, форма, характер материала) и использовать подходящие транспортные средства для перемещения.
5. В случае возникновения нештатных ситуаций или повреждений груза, сообщать об этом руководству и принять меры для их устранения.
6. Взаимодействовать с коллегами и другими участниками процесса, сотрудничая в выполнении общей задачи.
7. В случае необходимости, участвовать в обучении новых сотрудников, передавая им знания и опыт.
8. Не допускать неп

# Вывод уточняющего промта:

Проблема в том, что контекст не извлечен и вот она галюцинация!!!

# Борьба с галюцинациями

In [ ]:
# Импортируем необходимые классы из библиотеки llama_index
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.postprocessor import LLMRerank  # модуль реранжирования на базе LLM

# Создаем объект поискового движка из индекса
query_engine = index.as_query_engine(
    similarity_top_k=5,  # Указываем, что хотим получить топ-5 наиболее похожих результатов
    node_postprocessors=[
        LLMRerank(  # Используем LLMRerank для реранжирования результатов
            choice_batch_size=5,  # Размер батча для обработки
            top_n=2,  # Оставляем только топ-2 результата после реранжирования
        )
    ],
)

# Выполняем запрос к поисковому движку
response = query_engine.query(
    "Расскажи про должностные обзяности грузчика ? Опираясь на контекст документа, если сомвневаешься в ответе, то ничего не придумывай")

# Выводим ответ на запрос
print(response)

Ответственность грузчика:

1. За невыполнение и/или несвоейвременное, халатное выполнение своих должностних обязанностей.
2. За несоблюдение действующих инструкции, приказов и распоряжений по сохранении коммерческой тайны и конфиденциальной информацию.
3. За нарушение правил внутреннего трудоowego распорядка, трудовой дисциплины,правил техники безопасности и противопожарнной безопасности.


In [ ]:
# Уменьшаем размер чанков и настраиваем перекрытие
Settings.chunk_size = 256  # уменьшаем с 512 до 256
Settings.chunk_overlap = 64  # добавляем перекрытие

# Пробуем другую модель эмбеддингов
embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")
)


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [ ]:
# Создаем query_engine без LLMRerank
query_engine = index.as_query_engine(
    similarity_top_k=5
)

response = query_engine.query(
    "Расскажи про должностные обязанности грузчика. Используй только информацию из документа. Если какой-то пункт не описан в документе, не придумывай его. Структурируй ответ в виде нумерованного списка."
)


In [ ]:
print(response)

1. Осуществляет погрузку и разгрузку.
2. Производит очистку подвижного состаva после произведенной выгрузки груза.
3. Чистит и смазывает обслуживаемые подвижные составы и средства транспортировки.
4. Вносит на рассмотрение руководства предложения по совершенованию работы, связанной с предусмотренным настоящей инструкции обязанностями.
5. Требует от руководства предприятия обеспечения организационнотехнических условий и оформления установледных документов, необходимых для исполнение должностных обязанностей.


In [ ]:
from llama_index.core.postprocessor import LongContextReorder
reorder = LongContextReorder() # создаем экземпляр класса сортировщика
reorder_engine = index.as_query_engine(
    node_postprocessors=[reorder], similarity_top_k=10 # передаем сортировщика в постобработку
)

response = reorder_engine.query("Расскажи про должностные обязанности грузчика. Используй только информацию из документа. Если какой-то пункт не описан в документе, не придумывай его. Структурируй ответ в виде нумерованного списка")
print(response)

1. Погрузка и выгрузка грузова.
2. Очистка подвижного состава (подъемников, кранов, автокранов) после выгрузки груза.
3. Чистку и смазку обслуживаемых погрузочно-разгрузчных приспособлений и средств транспортирования.
4. Соблюдение правил и нормативов охраны труда, техники безопасности и противопожарной защиты.
5. Выполнение любых других работ, которые связаны с выполнением обязанностей, предусмотренных данным положением.


# Вывод по улучшению RAG

Удалось победить галяюцинации путями:

  * Улучшенный запрос с инструкциями
  * Настройкой параметров индексации - уменишил чанки, добавил перекрытие
  * Добавление другой модели эмбедингов, что я считаю и повлияло на улучшения ответа и победой на галюцинациями

Как видно выше, LLMRank нам ничего не дал, хотя бываил даже ошибки при выполнении запроса, а вывод запрсоа и вовсе не про то, что спрашивал, так что принял решение отключить LLMRank вовсе.

# Безопасность сотрудника, фильтрация запросов

In [ ]:
# Базовая фильтрация запросов
def validate_query(query):
    # Проверяем длину запроса
    if len(query.strip()) < 3:
        raise ValueError("Запрос слишком короткий")

    # Проверяем запрещенные слова
    banned_words = ["взломать", "украсть", "незаконный", "наркотики, личные данные"]
    if any(word in query.lower() for word in banned_words):
        raise ValueError("Недопустимый запрос")

    # Проверяем тип запроса
    if not isinstance(query, str):
        raise TypeError("Запрос должен быть строкой")

    return query


In [ ]:

# Создаем безопасный запрос
user_query = "Можно ли взломать вашу компанию?"

# Обрабатываем через фильтр
response = validate_query(user_query)

print(response)

ValueError: Недопустимый запрос

In [ ]:
# Задаю системный промпт
system_prompt = """
Ты — корпоративный ассистент.
* Отвечай только на основе предоставленного контекста
* Если информации нет — говори: "Данных нет"
* Не придумывай информацию
* Соблюдай деловой стиль общения
* Избегай сложных терминов без объяснения
"""


In [ ]:
# Улучшенный запрос с фильтрацией
def safe_query(query):
    try:
        # Валидируем запрос
        validated_query = validate_query(query)

        # Добавляем системный промпт
        response = query_engine.query(
            f"{system_prompt}\n\n{validated_query}"
        )

        return response

    except Exception as e:
        return f"Ошибка обработки запроса: {str(e)}"


In [ ]:
# Создаем безопасный запрос
user_query = "Расскажи про должностные обязанности грузчика"

# Обрабатываем через фильтр
response = safe_query(user_query)

print(response)


Грузчик отвечает за перемещение грузов между различными локациями, включая склад, транспортные средства и место назначения. Он должен быть квалифицированным рабочим, обладающим знаниями в области техники безопасности, правил транспортировки и хранения грузов. Грузчик также должен быть в состоянии работать в команде и координировать свои действия с другими сотрудниками.


# Трассировка запросов

In [ ]:
import logging  # Импортируем модуль для работы с логированием

# Настраиваем базовую конфигурацию логирования
# Уровень INFO означает, что будут записываться все сообщения уровня INFO и выше
logging.basicConfig(level=logging.INFO)

def log_query(query):
    """
    Функция для логирования входящего запроса

    Параметры:
    query (str) - входящий запрос для обработки

    Возвращает:
    str - оригинальный запрос после логирования
    """
    # Записываем информацию о полученном запросе в лог-файл
    # Используем форматированную строку для более читаемого вывода
    logging.info(f"Получен запрос: {query}")

    # Возвращаем оригинальный запрос для дальнейшей обработки
    return query

Функция ниже выполняет следующие задачи:

1. Сбор метрик - получает текущие показатели работы системы
2. Логирование данных - записывает основные метрики в лог-файл
3. Анализ производительности:
   * Проверяет общее количество запросов
   * Анализирует среднее время обработки
   * Оценивает процент ошибок
4. Система предупреждений:
   * Отслеживает превышение допустимого времени обработки (более 5 секунд)
   * Контролирует высокий уровень ошибок (более 5%)
5. Мониторинг состояния - помогает оперативно выявлять проблемы в работе системы

In [ ]:

def analyze_metrics():
    # Получаем текущие метрики из системы мониторинга
    metrics = monitoring.get_metrics()

    # Записываем информацию об анализе метрик в лог
    logging.info(f"Анализ метрик:")

    # Выводим основные показатели
    logging.info(f"Всего запросов: {metrics['total_requests']}")
    logging.info(f"Среднее время обработки: {metrics['avg_response_time']:.2f} сек")
    logging.info(f"Процент ошибок: {metrics['error_rate']*100:.2f}%")

    # Дополнительный анализ с предупреждениями
    # Проверяем время обработки запросов
    if metrics['avg_response_time'] > 5:
        logging.warning("Высокое время обработки запросов")

    # Проверяем процент ошибок
    if metrics['error_rate'] > 0.05:
        logging.warning("Высокий процент ошибок")


**Напишу класс для мониторнги с методами**

In [ ]:
import logging
from datetime import datetime
import time

# Класс для мониторинга запросов и ошибок
class Monitoring:
    def __init__(self):
        # Инициализация структуры метрик для хранения статистики
        self.metrics = {
            "requests": [],          # Список всех обработанных запросов
            "response_times": [],   # Время отклика каждого запроса
            "errors": 0              # Количество ошибок
        }

    # Метод для регистрации успешного запроса
    def log_request(self, prompt, response, start_time):
        try:
            # Если ответ является объектом с полем content, извлекаем содержимое
            if hasattr(response, 'content'):
                response_data = response.content.decode('utf-8')  # Преобразование байтов в строку
            # Если ответ имеет поле text, используем его непосредственно
            elif hasattr(response, 'text'):
                response_data = response.text
            # Иначе представляем ответ строкой
            else:
                response_data = str(response)

            # Вычисление времени обработки запроса
            end_time = time.time()       # Текущее время завершения обработки
            processing_time = end_time - start_time  # Продолжительность обработки

            # Добавляем запись о запросе в список запросов
            self.metrics["requests"].append({
                "prompt": prompt,           # Запрос пользователя
                "response": response_data, # Ответ системы
                "timestamp": datetime.now(), # Точное время обработки
                "processing_time": processing_time # Длительность обработки
            })

            # Сохранение времени обработки отдельно
            self.metrics["response_times"].append(processing_time)

            # Логирование успешности обработки запроса
            logging.info(f"Запрос: {prompt[:100]}...")      # Краткий вывод запроса
            logging.info(f"Ответ: {response_data[:100]}...") # Краткий вывод ответа
            logging.info(f"Время обработки: {processing_time:.2f} сек") # Информация о длительности обработки

        except Exception as e:
            # Регистрация ошибки при сохранении запроса
            logging.error(f"Ошибка при логировании: {str(e)}")

    # Метод для записи ошибки
    def log_error(self, error):
        self.metrics["errors"] += 1     # Увеличиваем счётчик ошибок
        logging.error(f"Ошибка: {str(error)}") # Логируем ошибку в журнал

    # Получение текущих метрик мониторинга
    def get_metrics(self):
        return {
            "total_requests": len(self.metrics["requests"]), # Всего запросов
            "avg_response_time": sum(self.metrics["response_times"]) / len(self.metrics["response_times"]) if self.metrics["response_times"] else 0, # Среднее время отклика
            "error_rate": self.metrics["errors"] / len(self.metrics["requests"]) if self.metrics["requests"] else 0 # Частота ошибок
        }

# Функция для обработки запроса с мониторингом
def process_query(prompt):
    start_time = time.time()  # Начало отсчета времени обработки
    try:
        # Выполняем обработку запроса движком
        response = query_engine.query(prompt)
        # Фиксируем успешный запрос
        monitoring.log_request(prompt, response, start_time)
        return response
    except Exception as e:
        # Если произошла ошибка, фиксируем её
        monitoring.log_error(e)
        return "Произошла ошибка при обработке запроса"


In [ ]:
# Пример использования
monitoring = Monitoring()

# Добавляем функцию вывода метрик
def show_metrics():
    metrics = monitoring.get_metrics()
    print("\n--- Метрики системы ---")
    print(f"Общее количество запросов: {metrics['total_requests']}")
    print(f"Среднее время обработки: {metrics['avg_response_time']:.2f} сек")
    print(f"Процент ошибок: {metrics['error_rate']*100:.2f}%")

    # Дополнительная статистика
    if metrics['total_requests'] > 0:
        print(f"Последнее время обработки: {monitoring.metrics['response_times'][-1]:.2f} сек")
        print(f"Всего ошибок: {monitoring.metrics['errors']}")
    else:
        print("Нет данных для отображения")

# Изменяем основной цикл для добавления возможности просмотра метрик

if __name__ == "__main__":
    while True:
        # Получаем ввод от пользователя
        user_input = input("\nВведите запрос (или 'метрики' для просмотра статистики) или 'Спасибо' для выхода: ")

        # Проверяем команду для просмотра метрик
        if user_input.lower() == 'метрики':
            show_metrics()
            continue  # Возвращаемся к началу цикла

        # Проверяем команду для выхода
        if user_input.lower() == 'спасибо':
            print("До свидания!")
            break  # Выходим из бесконечного цикла

        # Обрабатываем обычный запрос
        response = process_query(user_input)
        print(response)

        # Выполняем анализ метрик каждые 10 запросов
        if len(monitoring.metrics['requests']) % 10 == 0:
            analyze_metrics()




Введите запрос (или 'метрики' для просмотра статистики) или 'Спасибо' для выхода: Кто такой грузчик ? 
Грузчик - это профессия, в которой используется преимущвенно только мышечная физическая сила человека для выполнения различных рабочих операций по перемещению грузов.

Введите запрос (или 'метрики' для просмотра статистики) или 'Спасибо' для выхода: Расскажи про должностные обязанности кладовщика
Кладовщик выполняет следующие должnostные обязанности:

1. Осуществляет прием на склад, хранение и выдачу со склада различных матерiальных ценностей.
2. Проверяет соответствия принимаемым ценностям сопроводительным документам.
3. Перемещает материальные ценности на места хранения вручную или при помощи штабелеров или других механизмов с раскладкой (сортованием) их по видам, качеству, назначением и другим признакам.
4.

Кладовщик также должен знать законодательство, постановления, распоряжения, другие руководящие и нормативные материалы, касающиеся организации складского делопроизводства.

Вв

# Вывод

Ну что же, я вроде бы выполнил все требования задания. Кстати нашёл ещё одну галюцинацию, но это скорее, что нет уточняющего промтпа, про последний вопрос ответсвенность заведующего складом. И это супер, всегда есть над чем работать и даёт понимания, какая бы RAG система не была, в ней всегда есть место глюком и недочётам.

По поводу трассировки я не стал замарачиваться с фениксом, пошёл по пути наименьшего пути сопротевления и написал ф-цию по отслоеживанию метрик, для меня как для начинающего программиста этого очень даже, т.к даёт пониминие написания кода на языке Python

Файл txt я собирал сам из разных источников, делал максимально похожую инфу по всем должностям, что бы векторной базе данных было труднее понимать, что по смыслу находится ближе или дальше.  Думаю, что с графической базой данных, было бы легче извлекать контекст из файла.